In [1]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [2]:
apiKey = "fill_it_here"

In [3]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [4]:
dataset = pd.read_csv("/home/patrick/Downloads/test3label/keyphrases/done/keyphrases_SpotifyMixtral_Done.csv")

In [5]:
dataset

,Unnamed: 0,content,keyphrases_original,cluster,sentiment,count,keyphrases_reduced,Merged keyphrases_reduced,count merged keyphrases_reduced,kw newly generated,id Mkr,Merged keyphrases_reduced 2,count 2,kw newly generated 2,count 2 off,id Mkr 2
0,0,I was wrong with spotify I can't install it no...,"['user comment', 'Spotify App', 'installation ...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
1,688,I have had my account for three years. Now thi...,"['long-time Spotify user', 'account logout iss...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
2,810,(I have Premium) Randomly logged me out on bot...,"['Spotify Premium', 'logged out', 'phone', 'Xb...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
3,823,I'm paying for premium. You guys booted me off...,"['paying for premium', 'booted off premium', '...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
4,1314,Can't log in at all anymore. Can't seem to cha...,"[""Can't log in"", 'Login issue', 'Unable to cha...",49,0,7,"['Spotify app login issue', 'User account acce...","['Spotify app login issue', 'User account acce...",7,NaN,0,"['Spotify app login issue', 'User account acce...",7.0,NaN,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,2678,There's a bug where the now playing bar at the...,"['bug', 'now playing bar', 'bottom', 'music co...",298,1,2,"['Spotify app issues', 'Last update problems',...","['Spotify app issues', 'Last update problems',...",2,NaN,416,"['Spotify app issues', 'Last update problems',...",2.0,NaN,2,354
2696,2617,"Can't even play the song I want to play, let u...","[""can't play song"", 'freely control songs', 's...",469,0,1,"[""can't play song"", 'freely control songs', 's...","['Spotify app', 'used to harm people', 'can't ...",2,True,415,"['Spotify app', 'user comment', 'adds shows', ...",10.0,True,14,351
2697,2647,"Works good for me, I also found how to link yo...","['Spotify app functionality', 'user experience...",302,2,1,"['Spotify app functionality', 'user experience...","['Spotify app functionality', 'user experience...",2,True,417,"['Spotify app functionality', 'user experience...",2.0,NaN,2,355
2698,2682,"Bloated, unorganized, and a data-hog. Features...","['Bloated', 'unorganized', 'data-hog', 'playli...",224,0,1,"['Bloated', 'unorganized', 'data-hog', 'playli...","['playlists navigation', 'favorite artists nav...",1,True,418,"['Spotify app', 'user comment', 'adds shows', ...",10.0,True,14,351


In [6]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

sentiment
0    900
1    900
2    900
Name: count, dtype: int64


In [7]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]
train_data['row_number'] = train_data.index

/tmp/ipykernel_2803414/1289041381.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['row_number'] = train_data.index


In [8]:
train_data

,content,sentiment,Merged keyphrases_reduced 2,row_number
0,I was wrong with spotify I can't install it no...,0,"['Spotify app login issue', 'User account acce...",0
1,I have had my account for three years. Now thi...,0,"['Spotify app login issue', 'User account acce...",1
2,(I have Premium) Randomly logged me out on bot...,0,"['Spotify app login issue', 'User account acce...",2
3,I'm paying for premium. You guys booted me off...,0,"['Spotify app login issue', 'User account acce...",3
4,Can't log in at all anymore. Can't seem to cha...,0,"['Spotify app login issue', 'User account acce...",4
...,...,...,...,...
2695,There's a bug where the now playing bar at the...,1,"['Spotify app issues', 'Last update problems',...",2695
2696,"Can't even play the song I want to play, let u...",0,"['Spotify app', 'user comment', 'adds shows', ...",2696
2697,"Works good for me, I also found how to link yo...",2,"['Spotify app functionality', 'user experience...",2697
2698,"Bloated, unorganized, and a data-hog. Features...",0,"['Spotify app', 'user comment', 'adds shows', ...",2698


In [9]:
list = []
i = 0
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - negative, 2 - neutral, or 3 - positive. Here are examples to guide your classification:

                1. Comment: "I was wrong with spotify I can't install it now how dissapointing. Now, i can't log in even if my password is right."
                Topics / Keyphrases: ['Spotify app login issue', 'User account access problem', 'Incorrect password', 'Premium features login failure', 'User experience disappointment', 'Platform compatibility logout', 'Payment for premium login problem', 'Phone and Xbox login issue', 'Password update and reset', 'Account management failure']
                Label: 1
                
                2. Comment: "6 adds in a row all 30 seconds long because I did not want to buy the premium? Do better:|"
                Topics / Keyphrases: ['Spotify App issues', 'Google Play Store complaints', 'track display delay', '30-second ads', 'premium features limitations', 'broken app updates', 'song playback problems', 'song navigation restrictions', 'payment process errors', 'app functionality issues']
                Label: 1
                
                3. Comment: "Song Sharing doesn't work on Android 12, and lots of ads i hate this"
                Topics / Keyphrases: ['user experience', 'Spotify App', 'Google Play Store', 'user comments', 'song selection', 'ads', 'music variety', 'app analysis', '5 star review', 'offline mode']
                Label: 1                
                
                4. Comment: "Very buggy on Samsung Galaxy S20. Keeps playing with no way to stop it. Play controls will disappear and/ or become inactive. The only way to fix this is to force stop the app. This is only temporary and issue presents itself at least once a day."
                Topics / Keyphrases: ['Spotify app issues', 'music playback problems', 'user experience concerns', 'inactive play controls', 'app functionality bugs', 'song listing errors', 'queue management issues', 'app performance degradation', 'start-up screen failures', 'audio streaming interruptions']
                Label: 1
                
                5. Comment: "It's really irritating me.. There is no option for correct song Play on. Absolutely a damaze app..😠😠😠😠 Motherchod App is too more improvement needed.."
                Topics / Keyphrases: ['Spotify app', 'user experience', 'song playback', 'app improvement needed', 'enjoying music', 'repeated listening', 'discovery of appreciation', 'app issues', 'good tool for music', 'premium']
                Label: 1
                
                6. Comment: "I think its a good app i just wish i could chose the song i play"
                Topics / Keyphrases: ['user experience', 'Spotify App', 'Google Play Store', 'user comments', 'song selection', 'ads', 'music variety', 'app analysis', '5 star review', 'offline mode']
                Label: 2
                
                7. Comment: "It has mostly worked very well, but recently I am having trouble controlling what my app is playing because the player disappears without the music stopping, so I can play songs, but I can't pause and go back. Whenever this bug is fixed I'll update with a 5 star."
                Topics / Keyphrases: ['Spotify app issues', 'song playback interruptions', 'shuffle feature problem', 'manual song selection', 'playlist updating failure', 'no playing bar', 'song repetition', 'app stopping unexpectedly', 'incorrect playlist length', 'user experience issues']
                Label: 2
                                
                8. Comment: "App crashing problem was resolved but there needs to be some kinda of solution for it. I had to uninstall the app for about 2 weeks."
                Topics / Keyphrases: ['Spotify app issues', 'user experience frustration', 'playback problems', 'song glitching', 'app crashing', 'uninstalling app', 'subscription concerns', 'consecutive ads', 'persistent issues', 'technical improvement suggestions']
                Label: 2
                                                
                9. Comment: "This app is so nice l try every music app but on any app there is nothing related to kpop but in spotify yes there is every kpop band I can listen for free its really nice because I am a kpop fan and a real army also so this app is really very nice I love this app. Thanks for read my review Saraghe💜💜"
                Topics / Keyphrases: ['user's love for Spotify app', 'helpful for K-pop fans', 'new song availability', 'idols' songs on Spotify', 'using the app despite ads', 'Spotify features for K-pop fans', 'easy to use', 'variety of songs', 'positive user experience', 'recommends Spotify app']
                Label: 2                
                
                10. Comment: "Pretty good selection of music, and user friendly interface. But when I remove things from a queue and just let it play for any longer than an hour, and not always that long, it completely reshuffles the queue and it's infuriating"
                Topics / Keyphrases: ['Spotify App issues', 'user frustration', 'playback problems', 'music queue issues', 'app functionality', 'library organization', 'song recommendations', 'connectivity problems', 'playlist creation', 'podcast notifications']
                Label: 2

                11. Comment: "I highly recommend Spotify for those that love listening to their favorite music, as well as making their own playlists."
                Topics / Keyphrases: ['user's love for Spotify', 'personalized music recommendations', 'daily mixes', 'playlist creation', 'music discovery', 'Spotify features', 'user-friendly interface', 'music variety', 'app satisfaction', 'enjoyable music experience']
                Label: 3

                12. Comment: "Good music experience, really worth the premium subscription... Always there for me in every different moods"
                Topics / Keyphrases: ['music streaming', 'user satisfaction', 'Spotify App', 'positive experience', 'music variety', 'daily use', 'purchased subscription', 'initial feedback', 'playlist creation', 'Google Play Store comments']
                Label: 3
                
                13. Comment: "This app is very easy to use and all features are perfect and very understanding ,I always get all types of songs which are not available in other apps because I have use other apps before but after using Spotify I get the best app for music and nothing keep going. I purple 💜 U!!!"
                Topics / Keyphrases: ['user's love for Spotify app', 'helpful for K-pop fans', 'new song availability', 'idols' songs on Spotify', 'using the app despite ads', 'Spotify features for K-pop fans', 'easy to use', 'variety of songs', 'positive user experience', 'recommends Spotify app']
                Label: 3
                
                14. Comment: "Best app for music might use one of marvelous of alll"
                Topics / Keyphrases: ['Spotify app usage', 'music streaming experience', 'app analysis', 'user comments', 'positive feedback', 'sound quality', 'fantastic music app', 'content discovery', 'user interface', 'Hindi-English songs preference']
                Label: 3
                
                15. Comment: "ABSOLUTELY just brilliant the best thing since the old limewire days, love how Spotify recommend other artists, love how it's mobile then come home and have it blasting through the app on the telly. You have more control on the telly app than u do on your mobile, deffo worth a 5* rating without a doubt. If you love Allsorts of music, this is the app for you."
                Topics / Keyphrases: ['Spotify app', 'music enjoyment', 'playlists', 'limited skips', 'monthly cost', 'music lover', 'large music library', 'easy access to favorites', 'free version vs premium', 'genre variety']
                Label: 3

                
                Please provide the sentiment classification as a single number (1, 2, or 3) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)




I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - negative, 2 - neutral, or 3 - positive. Here are examples to guide your classification:

                1. Comment: "I was wrong with spotify I can't install it now how dissapointing. Now, i can't log in even if my password is right."
                Topics / Keyphrases: ['Spotify app login issue', 'User account access problem', 'Incorrect password', 'Premium features login failure', 'User experience disappointment', 'Platform compatibility logout', 'Payment for premium login problem', 'Phone and Xbox login issue', 'Password update and reset', 'Account management failure']
                Label: 1
                
                2. Comment: "6 adds in a row all 30 seconds long because I did not want to buy the premium? Do better:|"
                Topics / Keyphrases: ['Spotify App issues', 'Google Play Store complaints', 'track display d

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    string_message = f"""I need you to evaluate and assess the comment along with the set of topics/keyphrases, categorizing their sentiment into one of five levels: 1 - negative, 2 - neutral, or 3 - positive. Here are examples to guide your classification:

                1. Comment: "I was wrong with spotify I can't install it now how dissapointing. Now, i can't log in even if my password is right."
                Topics / Keyphrases: ['Spotify app login issue', 'User account access problem', 'Incorrect password', 'Premium features login failure', 'User experience disappointment', 'Platform compatibility logout', 'Payment for premium login problem', 'Phone and Xbox login issue', 'Password update and reset', 'Account management failure']
                Label: 1
                
                2. Comment: "6 adds in a row all 30 seconds long because I did not want to buy the premium? Do better:|"
                Topics / Keyphrases: ['Spotify App issues', 'Google Play Store complaints', 'track display delay', '30-second ads', 'premium features limitations', 'broken app updates', 'song playback problems', 'song navigation restrictions', 'payment process errors', 'app functionality issues']
                Label: 1
                
                3. Comment: "Song Sharing doesn't work on Android 12, and lots of ads i hate this"
                Topics / Keyphrases: ['user experience', 'Spotify App', 'Google Play Store', 'user comments', 'song selection', 'ads', 'music variety', 'app analysis', '5 star review', 'offline mode']
                Label: 1                
                
                4. Comment: "Very buggy on Samsung Galaxy S20. Keeps playing with no way to stop it. Play controls will disappear and/ or become inactive. The only way to fix this is to force stop the app. This is only temporary and issue presents itself at least once a day."
                Topics / Keyphrases: ['Spotify app issues', 'music playback problems', 'user experience concerns', 'inactive play controls', 'app functionality bugs', 'song listing errors', 'queue management issues', 'app performance degradation', 'start-up screen failures', 'audio streaming interruptions']
                Label: 1
                
                5. Comment: "It's really irritating me.. There is no option for correct song Play on. Absolutely a damaze app..😠😠😠😠 Motherchod App is too more improvement needed.."
                Topics / Keyphrases: ['Spotify app', 'user experience', 'song playback', 'app improvement needed', 'enjoying music', 'repeated listening', 'discovery of appreciation', 'app issues', 'good tool for music', 'premium']
                Label: 1
                
                6. Comment: "I think its a good app i just wish i could chose the song i play"
                Topics / Keyphrases: ['user experience', 'Spotify App', 'Google Play Store', 'user comments', 'song selection', 'ads', 'music variety', 'app analysis', '5 star review', 'offline mode']
                Label: 2
                
                7. Comment: "It has mostly worked very well, but recently I am having trouble controlling what my app is playing because the player disappears without the music stopping, so I can play songs, but I can't pause and go back. Whenever this bug is fixed I'll update with a 5 star."
                Topics / Keyphrases: ['Spotify app issues', 'song playback interruptions', 'shuffle feature problem', 'manual song selection', 'playlist updating failure', 'no playing bar', 'song repetition', 'app stopping unexpectedly', 'incorrect playlist length', 'user experience issues']
                Label: 2
                                
                8. Comment: "App crashing problem was resolved but there needs to be some kinda of solution for it. I had to uninstall the app for about 2 weeks."
                Topics / Keyphrases: ['Spotify app issues', 'user experience frustration', 'playback problems', 'song glitching', 'app crashing', 'uninstalling app', 'subscription concerns', 'consecutive ads', 'persistent issues', 'technical improvement suggestions']
                Label: 2
                                                
                9. Comment: "This app is so nice l try every music app but on any app there is nothing related to kpop but in spotify yes there is every kpop band I can listen for free its really nice because I am a kpop fan and a real army also so this app is really very nice I love this app. Thanks for read my review Saraghe💜💜"
                Topics / Keyphrases: ['user's love for Spotify app', 'helpful for K-pop fans', 'new song availability', 'idols' songs on Spotify', 'using the app despite ads', 'Spotify features for K-pop fans', 'easy to use', 'variety of songs', 'positive user experience', 'recommends Spotify app']
                Label: 2                
                
                10. Comment: "Pretty good selection of music, and user friendly interface. But when I remove things from a queue and just let it play for any longer than an hour, and not always that long, it completely reshuffles the queue and it's infuriating"
                Topics / Keyphrases: ['Spotify App issues', 'user frustration', 'playback problems', 'music queue issues', 'app functionality', 'library organization', 'song recommendations', 'connectivity problems', 'playlist creation', 'podcast notifications']
                Label: 2

                11. Comment: "I highly recommend Spotify for those that love listening to their favorite music, as well as making their own playlists."
                Topics / Keyphrases: ['user's love for Spotify', 'personalized music recommendations', 'daily mixes', 'playlist creation', 'music discovery', 'Spotify features', 'user-friendly interface', 'music variety', 'app satisfaction', 'enjoyable music experience']
                Label: 3

                12. Comment: "Good music experience, really worth the premium subscription... Always there for me in every different moods"
                Topics / Keyphrases: ['music streaming', 'user satisfaction', 'Spotify App', 'positive experience', 'music variety', 'daily use', 'purchased subscription', 'initial feedback', 'playlist creation', 'Google Play Store comments']
                Label: 3
                
                13. Comment: "This app is very easy to use and all features are perfect and very understanding ,I always get all types of songs which are not available in other apps because I have use other apps before but after using Spotify I get the best app for music and nothing keep going. I purple 💜 U!!!"
                Topics / Keyphrases: ['user's love for Spotify app', 'helpful for K-pop fans', 'new song availability', 'idols' songs on Spotify', 'using the app despite ads', 'Spotify features for K-pop fans', 'easy to use', 'variety of songs', 'positive user experience', 'recommends Spotify app']
                Label: 3
                
                14. Comment: "Best app for music might use one of marvelous of alll"
                Topics / Keyphrases: ['Spotify app usage', 'music streaming experience', 'app analysis', 'user comments', 'positive feedback', 'sound quality', 'fantastic music app', 'content discovery', 'user interface', 'Hindi-English songs preference']
                Label: 3
                
                15. Comment: "ABSOLUTELY just brilliant the best thing since the old limewire days, love how Spotify recommend other artists, love how it's mobile then come home and have it blasting through the app on the telly. You have more control on the telly app than u do on your mobile, deffo worth a 5* rating without a doubt. If you love Allsorts of music, this is the app for you."
                Topics / Keyphrases: ['Spotify app', 'music enjoyment', 'playlists', 'limited skips', 'monthly cost', 'music lover', 'large music library', 'easy access to favorites', 'free version vs premium', 'genre variety']
                Label: 3

                
                Please provide the sentiment classification as a single number (1, 2, or 3) and nothing more for the following comment:

                Comment: "{text}"
                Topics / Keyphrases: {topics}
                Label: ?
                """
    print("\n\n")
    print(string_message)
    print("\n")
    
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
# list.pop()

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = pd.Series(list)

In [ ]:
train_data

In [35]:
import re

def contains_number(s):
    return any(char.isdigit() for char in s)

def findNumbers(string):
    # Find all numbers in the string
    numbers = None
    if(contains_number(string)):
        numbers = re.findall(r'\d+', string)
    else:
        numbers = ""
    
    # Convert the numbers to integers
    
    numbers = [int(x) for x in numbers]
    
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    return x

In [36]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [39]:
train_data.to_csv('/home/patrick/Downloads/test3label/mixtralOutput/topics/filenameSpotifyFewShotMixtral-T.csv', index=False)

In [106]:
results = pd.read_csv("/home/patrick/Downloads/mistralOutput/filenameSpotifyFewShotMistral.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [109]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [111]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [113]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [114]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support